## 영화 리뷰 감정 분석

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets

In [2]:
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 40
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

cuda


In [4]:
# 토치텍스트 사용하여 필요한 설정
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

In [5]:
# IMDB 데이터셋
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

In [6]:
# 워드 임베딩에 필요한 단어 사전 생성
TEXT.build_vocab(trainset, min_freq=5)
LABEL.build_vocab(trainset)

In [7]:
# 배치 단위 학습
trainset, valset = trainset.split(split_ratio=0.8)
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (trainset, valset, testset),
    batch_size=BATCH_SIZE,
    shuffle=True,
    repeat=False)

In [8]:
vocab_size = len(TEXT.vocab)
n_classes = 2

In [9]:
print("[학습셋]: %d [검증셋]: %d [테스트셋]: %d [단어수]: %d [클래스] %d"
     % (len(trainset), len(valset), len(testset), vocab_size, n_classes))

[학습셋]: 20000 [검증셋]: 5000 [테스트셋]: 25000 [단어수]: 46159 [클래스] 2


In [15]:
## RNN 모델 구현
class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, 
                n_classes, dropout_p=0.2):
        super(BasicGRU, self).__init__()
        print("Building Basic GRU model...")
        
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embed_dim)
        
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)
        
    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0))
        x, _ = self.gru(x, h_0)
        h_t = x[:, -1, :]
        self.dropout(h_t)
        logit = self.out(h_t)
        return logit
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [16]:
def train(model, optimizer, train_iter):
        model.train()
        for b, batch in enumerate(train_iter):
            x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
            y.data.sub_(1)
            
            optimizer.zero_grad()
            logit = model(x)
            
            loss = F.cross_entropy(logit, y)
            loss.backward()
            optimizer.step()

In [17]:
def evaluate(model, val_iter):
        """evaluate model"""
        model.eval()
        corrects, total_loss = 0, 0
        for batch in val_iter:
            x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
            y.data.sub_(1)
            logit = model(x)
            loss = F.cross_entropy(logit, y, reduction='sum')
            total_loss += loss.item()
            corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
        
        size = len(val_iter.dataset)
        avg_loss = total_loss / size
        avg_accuracy = 100.0 * corrects / size
        return avg_loss, avg_accuracy

In [18]:
model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Building Basic GRU model...


In [20]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)
    
    print("[이폭: %d] 검증 오차:%5.2f | 검증 정확도:%5.2f" %
         (e, val_loss, val_accuracy))

[이폭: 1] 검증 오차: 0.69 | 검증 정확도:49.82
[이폭: 2] 검증 오차: 0.68 | 검증 정확도:56.56
[이폭: 3] 검증 오차: 0.65 | 검증 정확도:62.52
[이폭: 4] 검증 오차: 0.38 | 검증 정확도:83.46
[이폭: 5] 검증 오차: 0.34 | 검증 정확도:85.92
[이폭: 6] 검증 오차: 0.42 | 검증 정확도:84.98
[이폭: 7] 검증 오차: 0.42 | 검증 정확도:86.10
[이폭: 8] 검증 오차: 0.44 | 검증 정확도:84.76
[이폭: 9] 검증 오차: 0.44 | 검증 정확도:85.78
[이폭: 10] 검증 오차: 0.50 | 검증 정확도:86.26
[이폭: 11] 검증 오차: 0.51 | 검증 정확도:85.22
[이폭: 12] 검증 오차: 0.56 | 검증 정확도:85.30
[이폭: 13] 검증 오차: 0.54 | 검증 정확도:86.02
[이폭: 14] 검증 오차: 0.58 | 검증 정확도:85.94
[이폭: 15] 검증 오차: 0.57 | 검증 정확도:85.66
[이폭: 16] 검증 오차: 0.62 | 검증 정확도:85.76
[이폭: 17] 검증 오차: 0.59 | 검증 정확도:85.46
[이폭: 18] 검증 오차: 0.63 | 검증 정확도:85.92
[이폭: 19] 검증 오차: 0.66 | 검증 정확도:85.22
[이폭: 20] 검증 오차: 0.62 | 검증 정확도:86.00
[이폭: 21] 검증 오차: 0.63 | 검증 정확도:85.22
[이폭: 22] 검증 오차: 0.63 | 검증 정확도:85.82
[이폭: 23] 검증 오차: 0.70 | 검증 정확도:85.44
[이폭: 24] 검증 오차: 0.72 | 검증 정확도:85.96
[이폭: 25] 검증 오차: 0.76 | 검증 정확도:86.14
[이폭: 26] 검증 오차: 0.78 | 검증 정확도:85.96
[이폭: 27] 검증 오차: 0.80 | 검증 정확도:86.00
[이폭: 28] 검증 오차: 0.83 | 검증 정확도:85.94
[

In [21]:
# 검증오차가 가장 적은 최적의 모델을 저장
if not best_val_loss or val_loss < best_val_loss:
    if not os.path.isdir("snapshot"):
        os.makedirs("snapshot")
    torch.save(model.state_dict(), './snapshot/txtclassification.pt')
    best_val_loss = val_loss

In [22]:
model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차:%5.2f | 테스트 정확도:%5.2f' % (test_loss, test_acc))

테스트 오차: 0.87 | 테스트 정확도:85.67


## Seq2Seq 기계 번역

In [23]:
import torch
import torch.nn as nn
import random
import matplotlib.pyplot as plt

In [ ]:
vocab_size = 256 # 총 아스키 코드 개수

In [24]:
x_ = list(map(ord, "hello"))
y_ = list(map(ord, "hola"))
x = torch.LongTensor(x_)
y = torch.LongTensor(y_)

In [54]:
class Seq2Seq(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super(Seq2Seq, self).__init__()
        self.n_layers = 1
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.encoder = nn.GRU(hidden_size, hidden_size)
        self.decoder = nn.GRU(hidden_size, hidden_size)
        self.project = nn.Linear(hidden_size, vocab_size)

    def forward(self, inputs, targets):
        # 인코더에 들어갈 입력
        initial_state = self._init_state()
        embedding = self.embedding(inputs).unsqueeze(1)
        # embedding = [seq_len, batch_size, embedding_size]
        
        # 인코더 (Encoder)
        encoder_output, encoder_state = self.encoder(embedding, initial_state)
        # encoder_output = [seq_len, batch_size, hidden_size]
        # encoder_state  = [n_layers, seq_len, hidden_size]

        # 디코더에 들어갈 입력
        decoder_state = encoder_state
        decoder_input = torch.LongTensor([0])
        
        # 디코더 (Decoder)
        outputs = []
        
        for i in range(targets.size()[0]):
            decoder_input = self.embedding(decoder_input).unsqueeze(1)
            decoder_output, decoder_state = self.decoder(decoder_input, decoder_state)
            projection = self.project(decoder_output)
            outputs.append(projection)
            
            #티처 포싱(Teacher Forcing) 사용
            decoder_input = torch.LongTensor([targets[i]])

        outputs = torch.stack(outputs).squeeze()
        return outputs
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_size).zero_()

In [55]:
seq2seq = Seq2Seq(vocab_size, 16)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(seq2seq.parameters(), lr=1e-3)

In [56]:
# 학습
log = []
for i in range(1000):
    prediction = seq2seq(x, y)
    loss = criterion(prediction, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_val = loss.data
    log.append(loss_val)
    if i % 100 == 0:
        print("\n 반복:%d 오차: %s" % (i, loss_val.item()))
        _, top1 = prediction.data.topk(1, 1)
        print([chr(c) for c in top1.squeeze().numpy().tolist()])


 반복:0 오차: 10.839789390563965
['됽', '䐗', '낫', '蝀']

 반복:100 오차: 6.463189125061035
['h', 'h', 'h', 'o']

 반복:200 오차: 2.671907424926758
['h', 'h', 'h', 'a']

 반복:300 오차: 1.1441028118133545
['h', 'l', 'l', 'a']

 반복:400 오차: 0.7974412441253662
['h', 'l', 'l', 'a']

 반복:500 오차: 0.6793394684791565
['h', 'l', 'l', 'a']

 반복:600 오차: 0.6111288666725159
['h', 'l', 'l', 'a']

 반복:700 오차: 0.4535644054412842
['h', 'o', 'l', 'a']

 반복:800 오차: 0.34339526295661926
['h', 'o', 'l', 'a']

 반복:900 오차: 0.24392031133174896
['h', 'o', 'l', 'a']
